In [ ]:
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).
/gdrive


In [ ]:
%cd /gdrive/MyDrive/nlp-yuan_code/FinBERT-QA

/gdrive/MyDrive/nlp-yuan_code/FinBERT-QA


In [ ]:
from statistics import mean 

from src.process_data import *

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
data_path = '/gdrive/MyDrive/nlp-data/nlp-qa-datasets/FiQA/FiQA_train_task2/'
# Document id and Answer text
collection = load_answers_to_df(data_path+"FiQA_train_doc_final.tsv")
# Question id and Question text
queries = load_questions_to_df(data_path+"FiQA_train_question_final.tsv")
# Question id and Answer id pair
qid_docid = load_qid_docid_to_df(data_path+"FiQA_train_question_doc_final.tsv")

In [ ]:
!pip install pyserini
!pip install faiss

!apt install libomp-dev
!python -m pip install --upgrade faiss faiss-gpu
import faiss

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Reading package lists... Done
Building dependency tree       
Reading state information... Done
libomp-dev is already the newest version (5.0.1-1).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
#from pyserini.search import SimpleSearcher

import pandas as pd
import regex as re
import csv
from itertools import islice
import pickle
import numpy as np
import json
import os
import sys
import argparse
from pathlib import Path
from sklearn.model_selection import train_test_split
from pathlib import Path
# os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
#from pyserini.search as pysearch

from pyserini.search import SimpleSearcher

from src.utils import *

In [ ]:
fiqa_index = "/gdrive/MyDrive/nlp-yuan_code/FinBERT-QA/retriever/lucene-index-fiqa/"


def split_label(qid_docid):
    """
    Split question answer pairs into train, test, validation sets.

    Returns:
        train_label: Dictonary
            key - question id
            value - list of relevant docids
        test_label: Dictonary
            key - question id
            value - list of relevant docids
        valid_label: Dictonary
            key - question id
            value - list of relevant docids
    ----------
    Arguments:
        qid_docid: Dataframe containing the question id and relevant docids
    """
    # Group the answers for each question into a list
    qid_docid = qid_docid.groupby(['qid']).agg(lambda x: tuple(x)).applymap(list).reset_index()
    # Split data
    train, test_set = train_test_split(qid_docid, test_size=0.05)
    train_set, valid_set = train_test_split(train, test_size=0.1)
    # Expand the list of docids into individual rows to represent a single sample
    train_data = train_set.explode('docid')
    test_data = test_set.explode('docid')
    valid_data = valid_set.explode('docid')

    # Convert data into dictionary - key: qid, value: list of relevant docid
    train_label = label_to_dict(train_data)
    test_label = label_to_dict(test_data)
    valid_label = label_to_dict(valid_data)

    return train_label, test_label, valid_label

def split_question(train_label, test_label, valid_label, queries):
    """
    Split questions into train, test, validation sets.

    Returns:
        train_questions: Dataframe with qids
        test_questions: Dataframe with qids
        valid_questions: Dataframe with qids
    ----------
    Arguments:
        train_label: Dictionary contraining qid and list of relevant docid
        test_label: Dictionary contraining qid and list of relevant docid
        valid_label: Dictionary contraining qid and list of relevant docid
        queries: Dataframe containing the question id and question text
    """
    # Get a list of question ids
    train_q = list(train_label.keys())
    test_q = list(test_label.keys())
    valid_q = list(valid_label.keys())

    # Split question dataframe into train, test, valid set
    train_questions = queries[queries['qid'].isin(train_q)]
    test_questions = queries[queries['qid'].isin(test_q)]
    valid_questions = queries[queries['qid'].isin(valid_q)]

    return train_questions, test_questions, valid_questions

def split_label(qid_docid):
    """
    Split question answer pairs into train, test, validation sets.

    Returns:
        train_label: Dictonary
            key - question id
            value - list of relevant docids
        test_label: Dictonary
            key - question id
            value - list of relevant docids
        valid_label: Dictonary
            key - question id
            value - list of relevant docids
    ----------
    Arguments:
        qid_docid: Dataframe containing the question id and relevant docids
    """
    # Group the answers for each question into a list
    qid_docid = qid_docid.groupby(['qid']).agg(lambda x: tuple(x)).applymap(list).reset_index()
    # Split data
    train, test_set = train_test_split(qid_docid, test_size=0.05)
    train_set, valid_set = train_test_split(train, test_size=0.1)
    # Expand the list of docids into individual rows to represent a single sample
    train_data = train_set.explode('docid')
    test_data = test_set.explode('docid')
    valid_data = valid_set.explode('docid')

    # Convert data into dictionary - key: qid, value: list of relevant docid
    train_label = label_to_dict(train_data)
    test_label = label_to_dict(test_data)
    valid_label = label_to_dict(valid_data)

    return train_label, test_label, valid_label

def split_question(train_label, test_label, valid_label, queries):
    """
    Split questions into train, test, validation sets.

    Returns:
        train_questions: Dataframe with qids
        test_questions: Dataframe with qids
        valid_questions: Dataframe with qids
    ----------
    Arguments:
        train_label: Dictionary contraining qid and list of relevant docid
        test_label: Dictionary contraining qid and list of relevant docid
        valid_label: Dictionary contraining qid and list of relevant docid
        queries: Dataframe containing the question id and question text
    """
    # Get a list of question ids
    train_q = list(train_label.keys())
    test_q = list(test_label.keys())
    valid_q = list(valid_label.keys())

    # Split question dataframe into train, test, valid set
    train_questions = queries[queries['qid'].isin(train_q)]
    test_questions = queries[queries['qid'].isin(test_q)]
    valid_questions = queries[queries['qid'].isin(valid_q)]

    return train_questions, test_questions, valid_questions

def create_dataset(question_df, labels, cands_size):
    """Retrieves the top-k candidate answers for a question and
    creates a list of lists of the dataset containing the question id,
    list of relevant answer ids, and the list of answer candidates

    Returns:
        dataset: list of list in the form [qid, [pos ans], [ans candidates]]
    ----------
    Arguments:
        question_df: Dataframe containing the qid and question text
        labels: Dictonary containing the qid to text map
        cands_size: int - number of candidates to retrieve
    """
    dataset = []
    # Calls retriever
    searcher = SimpleSearcher(fiqa_index)
    # For each question
    for i, row in question_df.iterrows():
        qid = row['qid']
        tmp = []
        # Append qid
        tmp.append(qid)
        # Append list of relevant docs
        tmp.append(labels[qid])
        # Retrieves answer candidates
        cands = []
        query = row['question']
        query = re.sub('[£€§]', '', query)
        hits = searcher.search(query, k=cands_size)

        for docid in range(0, len(hits)):
            cands.append(int(hits[docid].docid))
        # Append candidate answers
        tmp.append(cands)
        dataset.append(tmp)

    return dataset

def get_dataset(query_path, labels_path, cands_size):
    """Splits the dataset into train, validation, and test set and creates
    the dataset form for training, validation, and testing.

    Returns:
        train_set: list of list in the form [qid, [pos ans], [ans candidates]]
        valid_set: list of list in the form [qid, [pos ans], [ans candidates]]
        test_set: list of list in the form [qid, [pos ans], [ans candidates]]
    ----------
    Arguments:
        query_path: str - path containing a list of qid and questions
        labels_path: str - path containing a list of qid and relevant docid
        cands_size: int - number of candidates to retrieve
    """
    # Question id and Question text
    queries = load_questions_to_df(query_path)
    # Question id and Answer id pair
    qid_docid = load_qid_docid_to_df(labels_path)
    # qid to docid label map
    labels = label_to_dict(qid_docid)
    train_label, test_label, valid_label = split_label(qid_docid)
    # Split Questions
    train_questions, test_questions, \
    valid_questions = split_question(train_label, test_label, valid_label, queries)

    print("\nGenerating training set...\n")
    train_set = create_dataset(train_questions, labels, cands_size)
    print("Generating validation set...\n")
    valid_set = create_dataset(valid_questions, labels, cands_size)
    print("Generating test set...\n")
    test_set = create_dataset(test_questions, labels, cands_size)

    return train_set, valid_set, test_set

def main():

    parser = argparse.ArgumentParser()
    # Required parameters
    parser.add_argument("--query_path", default=None, type=str, required=True,
    help="Path to the question id to text data in .tsv format. Each line should have at least two columns named (qid, question) separated by tab")
    parser.add_argument("--label_path", default=None, type=str, required=True,
    help="Path to the question id and answer id data in .tsv format. Each line should have at two columns named (qid, docid) separated by tab")

    # Optional parameters
    parser.add_argument("--cands_size", default=50, type=int, required=False,
    help="Number of candidates to retrieve per question.")
    parser.add_argument("--output_dir", default=Path.cwd()/'data/data_pickle/',
    type=str, required=False, help="The output directory where the generated data will be stored.")

    args = parser.parse_args()

    if len(sys.argv) < 4:
        print("Usage: python3 src/generate_data.py <query_path> <label_path>")
        sys.exit()

    train_set, valid_set, test_set = get_dataset(args.query_path, \
                                                 args.label_path, \
                                                 args.cands_size)

    save_pickle(args.output_dir + "train_set.pickle", train_set)
    save_pickle(args.output_dir + "valid_set.pickle", valid_set)
    save_pickle(args.output_dir + "test_set.pickle", test_set)

In [ ]:
query_path = "/gdrive/MyDrive/nlp-data/nlp-qa-datasets/FiQA/FiQA_train_task2/FiQA_train_question_final.tsv"
labels_path = "/gdrive/MyDrive/nlp-data/nlp-qa-datasets/FiQA/FiQA_train_task2/FiQA_train_question_doc_final.tsv"
train_set, valid_set, test_set = get_dataset(query_path, labels_path, 50)


Generating training set...

SimpleSearcher class has been deprecated, please use LuceneSearcher from pyserini.search.lucene instead
Generating validation set...

SimpleSearcher class has been deprecated, please use LuceneSearcher from pyserini.search.lucene instead
Generating test set...

SimpleSearcher class has been deprecated, please use LuceneSearcher from pyserini.search.lucene instead


In [ ]:
# Cleaning data
empty_docs, empty_id = get_empty_docs(collection)
# Remove empty answers from collection of answers
collection_cleaned = collection.drop(empty_id)
# Remove empty answers from qa pairs
qid_docid = qid_docid[~qid_docid['docid'].isin(empty_docs)]

print("Number of answers after cleaning: {}".format(len(collection_cleaned)))
print("Number of QA pairs after cleaning: {}".format(len(qid_docid)))

Number of answers after cleaning: 57600
Number of QA pairs after cleaning: 17072


In [ ]:
# Write collection df to file
save_tsv("retriever/collection_cleaned.tsv", collection_cleaned)

# Convert collection df to JSON file for Anserini's document indexer
collection_to_json("retriever/collection_json/docs.json", "retriever/collection_cleaned.tsv")

In [ ]:
processed_answers = process_answers(collection_cleaned)
processed_questions = process_questions(queries)

In [ ]:
word2index, word2count = create_vocab(processed_answers, processed_questions)

print("Vocab size: {}".format(len(word2index)))
print("Top {} common words: {}".format(35, Counter(word2count).most_common(35)))

Vocab size: 85034
Top 35 common words: [('the', 371203), ('to', 233559), ('a', 201620), ('you', 166702), ('and', 163066), ('of', 157574), ('is', 129894), ('in', 120019), ('that', 111416), ('for', 89366), ('it', 83822), ('i', 74100), ('your', 68153), ('are', 67255), ('if', 60689), ('be', 59266), ('on', 58382), ('have', 55754), ('as', 50088), ('this', 49868), ('not', 49227), ('or', 46080), ('with', 45894), ('they', 44485), ('but', 41690), ('can', 38863), ('will', 36865), ('at', 35548), ('an', 31392), ('money', 31003), ('so', 29980), ('$', 29096), ('would', 28750), ('from', 28582), ('more', 27378)]


In [ ]:
qid_to_text, docid_to_text = id_to_text(collection, queries)
qid_to_tokenized_text, docid_to_tokenized_text = id_to_tokenized_text(processed_answers, processed_questions)

In [ ]:
!pip install pydot --quiet
!pip install gensim==3.8.3 --quiet
!pip install tensorflow-datasets --quiet
!pip install -U tensorflow-text==2.8.2 --quiet
!pip install transformers --quiet
!pip install pydot --quiet

In [ ]:
import tensorflow as tf
from tensorflow import keras

from tensorflow.keras.layers import Embedding, Input, Dense, Lambda
from tensorflow.keras.models import Model
import tensorflow.keras.backend as K
import tensorflow_datasets as tfds
import tensorflow_text as tf_text

from transformers import BertTokenizer, TFBertModel


import sklearn as sk
import nltk
from nltk.corpus import reuters
from nltk.data import find

import matplotlib.pyplot as plt

import re

In [ ]:
bert_tokenizer = BertTokenizer.from_pretrained('bert-base-cased')

In [ ]:
from tqdm import tqdm


#bert_model = TFBertModel.from_pretrained('bert-base-cased')

def generate_bert_tokens(data):
  input_ids = []
  token_type_ids = []
  att_masks = []
  labels = []


  #for i, seq in enumerate(tqdm(train_set)):
  for i, seq in enumerate(data):
    qid, ans_labels, cands = seq[0], seq[1], seq[2]
    #print (qid, ans_labels, cands)
    
    # Map question id to text
    q_text = qid_to_text[qid]

    max_width = -1

    # For each answer in the candidates
    for docid in cands:
      # Map the docid to text
      ans_text = docid_to_text[docid] if docid_to_text.get(docid) else ''

      # Encode the sequence using BERT tokenizer
      encoded_seq = bert_tokenizer.encode_plus(q_text, ans_text[:500],
                                              max_length=512,
                                              pad_to_max_length=True,
                                              return_token_type_ids=True,
                                              truncation=True,
                                              return_attention_mask = True)

      #print(q_text, len(ans_text))

      # Get parameters
      input_ids.append(encoded_seq['input_ids'])
      token_type_ids.append(encoded_seq['token_type_ids'])
      att_masks.append(encoded_seq['attention_mask'])

      # If an answer is in the list of relevant answers assign
      # positive label
      labels.append(1 if docid in ans_labels else 0)


      #max_width = max(max_width, len(ans_text+q_text))
      #print(max_width)
  return input_ids, token_type_ids, att_masks, labels

In [ ]:
input_ids, token_type_ids, att_masks, labels = generate_bert_tokens(train_set[:1000])
save_pickle('/gdrive/MyDrive/nlp-yuan_code/FinBERT-QA/data/data_pickle/' + "input_ids1.pickle", input_ids)
save_pickle('/gdrive/MyDrive/nlp-yuan_code/FinBERT-QA/data/data_pickle/' + "token_type_ids1.pickle", token_type_ids)
save_pickle('/gdrive/MyDrive/nlp-yuan_code/FinBERT-QA/data/data_pickle/' + "att_masks1.pickle", att_masks)
save_pickle('/gdrive/MyDrive/nlp-yuan_code/FinBERT-QA/data/data_pickle/' + "labels1.pickle", labels)

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2307: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [ ]:
input_ids, token_type_ids, att_masks = generate_bert_tokens(train_set[1000:2000])
save_pickle('/gdrive/MyDrive/nlp-yuan_code/FinBERT-QA/data/data_pickle/' + "input_ids2.pickle", input_ids)
save_pickle('/gdrive/MyDrive/nlp-yuan_code/FinBERT-QA/data/data_pickle/' + "token_type_ids2.pickle", token_type_ids)
save_pickle('/gdrive/MyDrive/nlp-yuan_code/FinBERT-QA/data/data_pickle/' + "att_masks2.pickle", att_masks)

In [ ]:
input_ids, token_type_ids, att_masks = generate_bert_tokens(train_set[2000:3000])
save_pickle('/gdrive/MyDrive/nlp-yuan_code/FinBERT-QA/data/data_pickle/' + "input_ids3.pickle", input_ids)
save_pickle('/gdrive/MyDrive/nlp-yuan_code/FinBERT-QA/data/data_pickle/' + "token_type_ids3.pickle", token_type_ids)
save_pickle('/gdrive/MyDrive/nlp-yuan_code/FinBERT-QA/data/data_pickle/' + "att_masks3.pickle", att_masks)

In [ ]:
input_ids, token_type_ids, att_masks = generate_bert_tokens(train_set[3000:4000])
save_pickle('/gdrive/MyDrive/nlp-yuan_code/FinBERT-QA/data/data_pickle/' + "input_ids4.pickle", input_ids)
save_pickle('/gdrive/MyDrive/nlp-yuan_code/FinBERT-QA/data/data_pickle/' + "token_type_ids4.pickle", token_type_ids)
save_pickle('/gdrive/MyDrive/nlp-yuan_code/FinBERT-QA/data/data_pickle/' + "att_masks4.pickle", att_masks)

In [ ]:
input_ids, token_type_ids, att_masks = generate_bert_tokens(train_set[4000:])
save_pickle('/gdrive/MyDrive/nlp-yuan_code/FinBERT-QA/data/data_pickle/' + "input_ids5.pickle", input_ids)
save_pickle('/gdrive/MyDrive/nlp-yuan_code/FinBERT-QA/data/data_pickle/' + "token_type_ids5.pickle", token_type_ids)
save_pickle('/gdrive/MyDrive/nlp-yuan_code/FinBERT-QA/data/data_pickle/' + "att_masks5.pickle", att_masks)

In [ ]:
save_pickle('/gdrive/MyDrive/nlp-yuan_code/FinBERT-QA/data/data_pickle/' + "input_ids5.pickle", input_ids)
save_pickle('/gdrive/MyDrive/nlp-yuan_code/FinBERT-QA/data/data_pickle/' + "token_type_ids5.pickle", token_type_ids)
save_pickle('/gdrive/MyDrive/nlp-yuan_code/FinBERT-QA/data/data_pickle/' + "att_masks5.pickle", att_masks)

In [ ]:
id1 = load_pickle('/gdrive/MyDrive/nlp-yuan_code/FinBERT-QA/data/data_pickle/' + "input_ids1.pickle")
type1 = load_pickle('/gdrive/MyDrive/nlp-yuan_code/FinBERT-QA/data/data_pickle/' + "token_type_ids1.pickle")
mask1 = load_pickle('/gdrive/MyDrive/nlp-yuan_code/FinBERT-QA/data/data_pickle/' + "att_masks1.pickle")
label1 = load_pickle('/gdrive/MyDrive/nlp-yuan_code/FinBERT-QA/data/data_pickle/' + "labels1.pickle")
print(len(id1), len(type1), len(mask1), len(label1))

49979 49979 49979 49979


In [ ]:
save_pickle('/gdrive/MyDrive/nlp-yuan_code/FinBERT-QA/data/data_pickle/' + "train_set.pickle", train_set)
save_pickle('/gdrive/MyDrive/nlp-yuan_code/FinBERT-QA/data/data_pickle/' + "valid_set.pickle", valid_set)
save_pickle('/gdrive/MyDrive/nlp-yuan_code/FinBERT-QA/data/data_pickle/' + "test_set.pickle", test_set)

In [ ]:
#train_dataloader = get_dataloader(train_set, "train")

In [ ]:
# Question id and Question text
#query_path = "/gdrive/MyDrive/nlp-data/nlp-qa-datasets/FiQA/FiQA_train_task2/FiQA_train_question_final.tsv"
#queries = load_questions_to_df(query_path)
# Question id and Answer id pair
#labels_path = "/gdrive/MyDrive/nlp-data/nlp-qa-datasets/FiQA/FiQA_train_task2/FiQA_train_question_doc_final.tsv"
#qid_docid = load_qid_docid_to_df(labels_path)
# qid to docid label map
#labels = label_to_dict(qid_docid)
#train_label, test_label, valid_label = split_label(qid_docid)
# Split Questions
#train_questions, test_questions, \
#valid_questions = split_question(train_label, test_label, valid_label, queries)

In [ ]:
def create_bert_model():
    """
    Build a simple classification model with BERT. Use the CLS Token output for classification purposes.
    """
    
    ### YOUR CODE HERE
    hidden_size = 100
    dropout = 0.3
    learning_rate= 0.00005
    bert_model = TFBertModel.from_pretrained('bert-base-cased')

    #restrict training to the train_layers outer transformer layers
    input_ids = tf.keras.layers.Input(shape=(512,), dtype=tf.int64, name='input_ids_layer') #--SOLUTION--
    token_type_ids = tf.keras.layers.Input(shape=(512,), dtype=tf.int64, name='token_type_ids_layer')
    attention_mask = tf.keras.layers.Input(shape=(512,), dtype=tf.int64, name='attention_mask_layer')

    bert_inputs = {'input_ids': input_ids,
                   'token_type_ids': token_type_ids,
                   'attention_mask': attention_mask}         

    bert_out = bert_model(bert_inputs) 

    cls_token = bert_out[0][:,0,:]
    #cls_token = bert_out[1]

    hidden = tf.keras.layers.Dense(hidden_size, activation='relu', name='hidden_layer')(cls_token)
    hidden = tf.keras.layers.Dropout(dropout)(hidden)  

    classification = tf.keras.layers.Dense(1, activation='sigmoid',name='classification_layer')(hidden)

    
    classification_model = tf.keras.Model(inputs=[input_ids, token_type_ids, attention_mask], outputs=[classification])
    
    classification_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
                            loss=tf.keras.losses.BinaryCrossentropy(from_logits=False), 
                            metrics='accuracy') 
    ### END YOUR CODE
    
    return classification_model

In [ ]:
model = create_bert_model()

Some layers from the model checkpoint at bert-base-cased were not used when initializing TFBertModel: ['mlm___cls', 'nsp___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-cased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


In [ ]:
del input_ids
del token_type_ids
del att_masks
del labels

NameError: ignored

In [ ]:
n = len(id1[0])
for i in range(len(id1)):
  if len(id1[i]) != n:
    print(i, len(id1[i]))
    break
print(n)

n = len(type1[0])
for i in range(len(type1)):
  if len(type1[i]) != n:
    print(i, len(type1[i]))
    break
print(n)

n = len(mask1[0])
for i in range(len(mask1)):
  if len(mask1[i]) != n:
    print(i, len(mask1[i]))
    break
print(n)

n = len(label1)


2500
2500
2500


In [ ]:
bert_avg_model_history = model.fit([np.array(id1[:100]), np.array(type1[:100]), np.array(mask1[:100])],np.array(label1[:100]),
                                   validation_data=([np.array(id1[:10]), np.array(type1[:10]), np.array(mask1[:10])],np.array(label1[:10])),
                                   batch_size=8,epochs=1)  

13/13 [==============================] - 381s 27s/step - loss: 0.2313 - accuracy: 0.9100 - val_loss: 0.0151 - val_accuracy: 1.0000
